# Acquire

- https://www.latimes.com/california/story/2020-03-16/los-angeles-parking-ticket-street-sweeping-coronavirus-covid19
- https://www.latimes.com/california/story/2020-10-15/street-sweeping-parking-enforcement-resumes-today
- https://abc7.com/society/las-resumed-parking-enforcement-prompts-outcry/7079278/

- https://www.theeastsiderla.com/site/about_the_eastsider/
- https://xtown.la/2020/10/15/parking-ticket-los-angeles/

In [1]:
import numpy as np
import pandas as pd
import os
import spacy

from pdfminer.high_level import extract_text
from transformers import pipeline

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
# Allocate a pipeline for sentiment-analysis
classifier = pipeline('sentiment-analysis')
classifier('We are very happy to include pipeline into the transformers repository.')

[{'label': 'POSITIVE', 'score': 0.9978193640708923}]

In [3]:
# Allocate a pipeline for question-answering
question_answerer = pipeline('question-answering')
question_answerer({
    'question': 'What is the name of the repository ?',
    'context': 'Pipeline have been included in the huggingface/transformers repository'
})

{'score': 0.5074627943662177,
 'start': 35,
 'end': 59,
 'answer': 'huggingface/transformers'}

In [3]:
# Load in the text from a pdf using pdfminer
extract_text('city-documents/city-council/LADOT-transition-plan.pdf')[:500]

'CITY OF LOS ANGELES\nINTER-DEPARTMENTAL MEMORANDUM\n\nDate:\n\nSeptember 17, 2020\n\nTo:\n\nHonorable City Council\nc/o City Clerk, Room 395, City Hall\nAttention: Honorable Mike Bonin, Chair, Transportation Committee\n\nFrom:\n\nSeleta J. Reynolds, General Manager ^ \nDepartment of Transportation\n\nSubject:\n\nTRANSITION PLAN TO RESUME PARKING ENFORCEMENT FOR PREVIOUSLY SUSPENDED \n\nPARKING INFRACTIONS AND PROPOSED ECONOMIC RELIEF MEASURES FOR PARKING \n\nFINES\n\nSUMMARY\n\nThe Los Angeles Department of Transportation '

In [4]:
# Create an empty dictionary to scrape text from all the pdfs
# stored in the city-documents folder.
documents = []

for root, dirs, files in os.walk("city-documents/"):
    for file in files:
        # If the ends with .pdf, display the path
        if file.endswith(".pdf"):
            print(os.path.join(root, file))
            # Add the filename and relative path as a dictionary to documents
            documents.append({'pdf_name': file,
                              'path': os.path.join(root, file)})

city-documents/city-council/LADOT-transition-plan.pdf
city-documents/city-council/public-outreach-period.pdf
city-documents/city-council/relief-report-motion.pdf
city-documents/city-council/relief-program-report-121720.pdf
city-documents/public-comments/public-comments-parking-enforcement.pdf
city-documents/LADOT/enforcement.pdf
city-documents/LADOT/citation-pay-program.pdf


In [5]:
# Convert the list of dictionaries into a dataframe
pdfs = pd.DataFrame(documents)
pdfs

,pdf_name,path
0,LADOT-transition-plan.pdf,city-documents/city-council/LADOT-transition-p...
1,public-outreach-period.pdf,city-documents/city-council/public-outreach-pe...
2,relief-report-motion.pdf,city-documents/city-council/relief-report-moti...
3,relief-program-report-121720.pdf,city-documents/city-council/relief-program-rep...
4,public-comments-parking-enforcement.pdf,city-documents/public-comments/public-comments...
5,enforcement.pdf,city-documents/LADOT/enforcement.pdf
6,citation-pay-program.pdf,city-documents/LADOT/citation-pay-program.pdf


In [6]:
# Collect the relative paths for each pdfs
pdfs_to_scrape = []

for index, file in pdfs.iterrows():
    print(file['path'])
    pdfs_to_scrape.append(file['path'])

city-documents/city-council/LADOT-transition-plan.pdf
city-documents/city-council/public-outreach-period.pdf
city-documents/city-council/relief-report-motion.pdf
city-documents/city-council/relief-program-report-121720.pdf
city-documents/public-comments/public-comments-parking-enforcement.pdf
city-documents/LADOT/enforcement.pdf
city-documents/LADOT/citation-pay-program.pdf


In [7]:
# Display all file paths
pdfs_to_scrape

['city-documents/city-council/LADOT-transition-plan.pdf',
 'city-documents/city-council/public-outreach-period.pdf',
 'city-documents/city-council/relief-report-motion.pdf',
 'city-documents/city-council/relief-program-report-121720.pdf',
 'city-documents/public-comments/public-comments-parking-enforcement.pdf',
 'city-documents/LADOT/enforcement.pdf',
 'city-documents/LADOT/citation-pay-program.pdf']

In [8]:
# Create an empty list to store text extracted from each pdf.
text = []

# Scrape the text from each pdf and store the result in text
for file in pdfs_to_scrape:
    text.append(extract_text(file))

In [9]:
# Display the number of documents stored in the variable `text`.
len(text)

7

In [10]:
# Display text from the first document
print(text[0][:200])

CITY OF LOS ANGELES
INTER-DEPARTMENTAL MEMORANDUM

Date:

September 17, 2020

To:

Honorable City Council
c/o City Clerk, Room 395, City Hall
Attention: Honorable Mike Bonin, Chair, Transportation Com


In [12]:
# Display the first 200 characters of each docuemnt in the variable `text`.
for i in range(0, len(text)):
    print(f"\nDOCUMENT #{i+1}")
    print("-----------------")
    print(text[i][:20].strip())


DOCUMENT #1
-----------------
CITY OF LOS ANGELES

DOCUMENT #2
-----------------
MOTION

3 0 A

I MOV

DOCUMENT #3
-----------------
TRANSPORTATION

MOTI

DOCUMENT #4
-----------------
File No. 20-1365

TR

DOCUMENT #5
-----------------
Communication from P

DOCUMENT #6
-----------------
FOR IMMEDI

DOCUMENT #7
-----------------
FOR IMMEDI


In [28]:
# Load the large English NLP model
nlp = spacy.load('en_core_web_lg')

# The text we want to examine

# Parse the text with spaCy. This runs the entire pipeline.
doc = nlp(text[0])

# 'doc' now contains a parsed version of text. We can use it to do anything we want!
# For example, this will print out all the named entities that were detected:
for entity in doc.ents:
    print(f"{entity.text:<25} ({entity.label_})")

CITY OF                   (ORG)
LOS ANGELES               (GPE)
September 17, 2020        (DATE)
City Council              (ORG)
395                       (CARDINAL)
City Hall
                (FAC)
Mike Bonin                (PERSON)
Transportation Committee

 (ORG)
Seleta J. Reynolds        (PERSON)
Department of Transportation

 (ORG)
The Los Angeles Department of Transportation (ORG)
LADOT                     (ORG)
CF 20-0147-S7             (ORG)
the City Council          (ORG)
1                         (CARDINAL)
LADOT                     (ORG)
October 1, 2020           (DATE)
overnight                 (TIME)
2                         (CARDINAL)
LADOT                     (ORG)
October 15, 2020          (DATE)
3                         (CARDINAL)
LADOT                     (ORG)
October 22, 2020          (DATE)
4                         (CARDINAL)
LADOT                     (ORG)
January 1, 2021           (DATE)
March 4, 2020             (DATE)
the City Council          (ORG)
the Decla

In [54]:
public_comments = text[4].replace("\xa0", ' ')
public_comments = public_comments.replace("\n", ' ')

In [61]:
doc = nlp(public_comments)

filtered_tokens = [token for token in doc if not token.is_stop]

In [62]:
filtered_tokens

[Communication,
 Public,
      ,
 :,
 Date,
 Submitted,
 :,
 Council,
 File,
 :,
 Comments,
 Public,
 Posting,
 :,
  ,
 comments,
 reference,
 item,
 #,
 3,
 today,
  ,
 Armen,
 Makasjian,
 08/17/2020,
 01:30,
 PM,
 20,
 -,
 0147,
 -,
 S7,
   ,
 Transportation,
 Committee,
 Meeting,
 .,
 important,
 street,
 sweeping,
 continue,
 neighborhoods,
 .,
 people,
 sleeping,
 cars,
 parked,
 weeks,
 .,
 North,
 Berendo,
 St.,
 north,
 Hollywood,
 Blvd,
 .,
 ,,
 vehicles,
 ,,
 including,
 large,
 RV,
 parked,
 weeks,
 moving,
 .,
 resulted,
 infestation,
 rats,
 vermin,
 ,,
 including,
 roaches,
 .,
 important,
 city,
 continue,
 street,
 sweeping,
 prevent,
 blight,
 unsanitary,
 conditions,
 .,
 reconsider,
 motion,
 CONTINUE,
 street,
 sweeping,
 .,
 Thank,
 !,
   ,
 Communication,
 Public,
  ,
 Scott,
 P.,
 09/28/2020,
 09:01,
 PM,
 20,
 -,
 0147,
 -,
 S7,
       ,
 :,
 Date,
 Submitted,
 :,
 Council,
 File,
 :,
 Comments,
 Public,
 Posting,
 :,
  ,
 Scott,
 live,
 El,
 Sereno,
 want,
 enc

In [13]:
# from spacy import displacy

# sentence = text[0]
# sentence_nlp = nlp(sentence)

# # visualize named entities
# displacy.render(sentence_nlp, style='ent', jupyter=True)